# Python을 활용한 웹 정보 수집

## urllib 패키지

* `urlencode` : URL 인수 문자열 생성
* `urlopen` : 웹서버 연결
* `urlretrieve` : 웹서버 연결 및 HTML 문서 저장

In [40]:
import urllib

In [45]:
params = urllib.urlencode({"a": 4, "b": 20, "c": 2016, "d": 6, "e": 30, "f": 2016, "s": "^KS11"})
params

'a=4&c=2016&b=20&e=30&d=6&f=2016&s=%5EKS11'

In [70]:
url = 'http://ichart.finance.yahoo.com/table.csv?g=d&ignore=.csv&%s' % params
data = urllib.urlopen(url).read()
print(data)

Date,Open,High,Low,Close,Volume,Adj Close
2016-07-06,1980.109985,1982.800049,1944.329956,1953.119995,457400,1953.119995
2016-07-05,1995.689941,1996.77002,1985.939941,1989.849976,405500,1989.849976
2016-07-04,1989.439941,1997.369995,1985.109985,1995.300049,468600,1995.300049
2016-07-01,1977.359985,1993.97998,1975.890015,1987.319946,448100,1987.319946
2016-06-30,1971.540039,1972.810059,1959.280029,1970.349976,349900,1970.349976
2016-06-29,1946.310059,1966.209961,1941.900024,1956.359985,451200,1956.359985
2016-06-28,1907.709961,1937.98999,1907.48999,1936.219971,373500,1936.219971
2016-06-27,1901.849976,1926.869995,1900.829956,1926.849976,394300,1926.849976
2016-06-24,2001.550049,2001.550049,1892.75,1925.23999,750500,1925.23999
2016-06-23,1989.420044,1990.709961,1980.140015,1986.709961,455400,1986.709961
2016-06-22,1981.780029,1994.76001,1979.369995,1992.579956,407400,1992.579956
2016-06-21,1977.800049,1983.869995,1971.98999,1982.699951,557400,1982.699951
2016-06-20,1974.430054,1984.069946

In [58]:
%cd /home/dockeruser
urllib.urlretrieve(url, "/home/dockeruser/kospi.csv")
%ls

/home/dockeruser
anaconda2/  data/  kospi.csv  notebooks/  scikit_learn_data/


In [59]:
csv = open("/home/dockeruser/kospi.csv")
csv.readlines()[:5]

['Date,Open,High,Low,Close,Volume,Adj Close\n',
 '2016-07-06,1980.109985,1982.800049,1944.329956,1953.119995,457400,1953.119995\n',
 '2016-07-05,1995.689941,1996.77002,1985.939941,1989.849976,405500,1989.849976\n',
 '2016-07-04,1989.439941,1997.369995,1985.109985,1995.300049,468600,1995.300049\n',
 '2016-07-01,1977.359985,1993.97998,1975.890015,1987.319946,448100,1987.319946\n']

## requests 패키지

* http://docs.python-requests.org/en/master/
* HTTP protocols (get, post, put, delete, head, options)


https://www.google.com/finance/historical?q=KRX%3AKOSPI200

In [242]:
import requests
url = "https://www.google.com/finance/historical?q=KRX%3AKOSPI200"
req = requests.get(url)
print(req.text[3000:4000])

Lrg-RaWvg" class=invfr tabindex="-1"></iframe><div id=gbar><nobr><a class=gb1 href="https://www.google.co.kr/webhp?tab=ew">Search</a> <a class=gb1 href="http://www.google.co.kr/imghp?hl=en&tab=ei">Images</a> <a class=gb1 href="https://maps.google.co.kr/maps?hl=en&tab=el">Maps</a> <a class=gb1 href="https://play.google.com/?hl=en&tab=e8">Play</a> <a class=gb1 href="https://www.youtube.com/?tab=e1">YouTube</a> <a class=gb1 href="https://mail.google.com/mail/?tab=em">Gmail</a> <a class=gb1 href="https://drive.google.com/?tab=eo">Drive</a> <a class=gb1 href="https://www.google.com/calendar?tab=ec">Calendar</a> <a class=gb1 style="text-decoration:none" href="https://www.google.co.kr/intl/en/options/"><u>More</u> &raquo;</a></nobr></div><div id=guser width=100%><nobr><span id=gbn class=gbi></span><span id=gbf class=gbf></span><span id=gbe></span><a  href="http://www.google.com/support/finance?hl=en" class=gb4>Help</a> | <a target=_top id=gb_70 href="https://accounts.google.com/ServiceLogin?s

## beautifulsoup 패키지

* https://www.crummy.com/software/BeautifulSoup/bs4/doc/
* HTML 문서 파싱 및 태그 검색

In [300]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(req.text, 'lxml')

In [305]:
import dateutil

list_records = []
table = soup.find("table", class_="historical_price")
for i, r in enumerate(table.find_all('tr')):
    for j, c in enumerate(r.find_all('td')):
        if j == 0:
            record = {"date": dateutil.parser.parse(c.text.strip())}
        elif j == 1:
            record.update({"open": float(c.text.strip())})
        elif j == 2:
            record.update({"high": float(c.text.strip())})
        elif j == 3:
            record.update({"low": float(c.text.strip())})
        elif j == 4:
            record.update({"close": float(c.text.strip())})
        elif j == 5:
            record.update({"volume": int(c.text.strip().replace(',',''))})
    list_records.append(record)

In [306]:
list_records[:5]

[{'close': 240.91,
  'date': datetime.datetime(2016, 5, 25, 0, 0),
  'high': 241.45,
  'low': 239.28,
  'open': 239.68,
  'volume': 91189000},
 {'close': 241.86,
  'date': datetime.datetime(2016, 7, 6, 0, 0),
  'high': 245.74,
  'low': 240.73,
  'open': 245.37,
  'volume': 71672000},
 {'close': 246.91,
  'date': datetime.datetime(2016, 7, 5, 0, 0),
  'high': 247.84,
  'low': 246.51,
  'open': 247.67,
  'volume': 54810000},
 {'close': 247.62,
  'date': datetime.datetime(2016, 7, 4, 0, 0),
  'high': 247.94,
  'low': 246.16,
  'open': 246.66,
  'volume': 63633000},
 {'close': 246.52,
  'date': datetime.datetime(2016, 7, 1, 0, 0),
  'high': 247.57,
  'low': 244.78,
  'open': 244.96,
  'volume': 63045000}]

In [307]:
df = pd.DataFrame(list_records,
                  columns=["date", "open", "high", "low", "close", "volume"])
df.tail()

,date,open,high,low,close,volume
26,2016-05-31,241.03,243.84,240.36,243.63,117966000
27,2016-05-30,241.94,242.07,240.41,241.73,67446000
28,2016-05-27,241.25,242.41,240.74,241.85,92225000
29,2016-05-26,241.56,242.05,240.45,240.58,130282000
30,2016-05-25,239.68,241.45,239.28,240.91,91189000


## lxml 패키지

* http://lxml.de/index.html
* xpath 사용 가능
 * https://en.wikipedia.org/wiki/XPath

In [308]:
import lxml.html 
tree = lxml.html.fromstring(req.text)

In [309]:
import dateutil
dates = [dateutil.parser.parse(x.text.strip()) for x in tree.xpath('//td[@class="lm"]')]
volumes = np.array([int(x.text.strip().replace(',','')) for x in tree.xpath('//td[@class="rgt rm"]')])
prices = np.reshape([float(x.text.strip()) for x in tree.xpath('//td[@class="rgt"]')], (-1, 4))
price_o = prices[:,0]
price_h = prices[:,1]
price_l = prices[:,2]
price_c = prices[:,3]

In [310]:
df = pd.DataFrame({"date": dates, "open": price_o, "high": price_h, "low": price_l, "close": price_c, "volume": volumes},
                  columns=["date", "open", "high", "low", "close", "volume"])
df.tail()

,date,open,high,low,close,volume
25,2016-05-31,241.03,243.84,240.36,243.63,117966000
26,2016-05-30,241.94,242.07,240.41,241.73,67446000
27,2016-05-27,241.25,242.41,240.74,241.85,92225000
28,2016-05-26,241.56,242.05,240.45,240.58,130282000
29,2016-05-25,239.68,241.45,239.28,240.91,91189000
